# Facebook Public Data Analysis and Classification Using CNN in TensorFlow  
INFO 7390---Advances Data Sci/Architecture SEC-04-Spring 2018  

## Introduction

Social media has become a very significant open communication medium currently. This has motivated a lot research on social media data analysis from different aspects using machine learning techniques. Consider the problem of evaluating an Facebook official account, our group frame the problem as collecting dataset and model training. During the progress we extract more than 30,000 pieces of data from 10 public accounts. On the other hands we propose a model combining CNN (convolutional neutral network), correlation and classification to extract
features and patterns related social media posts and use them to derive insights about the sources and official account who generated the post with 89.10% accuracy on test dataset. Realize the program’s initial understanding of a public homepage and be able to make further predictions.

### Flow chart

The following chart shows roughly how the data flows in our final project that is implemented below.

![my_test_image](Picture/p1.jpeg)

Since Facebook has a very strict rule among user privacy policy, we can only obtain information published in our own account or information released by official account. Therefore we’ll only focus on public data.
Utilizing Facebook graph API services to collect data from different official accounts. The raw data we collected consists of two parts: text and account name. In this part we also need access a token to access graph API. Original dataset, as known as target came serialized in JSON which we acquired by a three-step loop and then performed all exploratory data from this section. In fact, there exist a huge amount of posts are merely forwarded and do not have their own textual information. In this case we forced to jump out of loop to maintain data acquisition.
A same method is also used to process pictures. We use google vision API to deal with image content. It provides powerful Image analytics capabilities and enables us to build the next processing. API can see and understand the content within the images and finally return key
words which can describe the images forward us.
To make data suitable for further classify and analysis, before convolutional neutral network digesting we completed bellowing pre-processing:1.Converted all characters to lower cases.2.Remove punctuations and irrelevant symbols. 3.Remove all blanks via text field. Integrate pre-processed text information into a csv file. This csv file contains label and text and shown as ‘industrial raw material’ in the picture below. Processing through 'factory-CNN'we will reach classification model.

## Implementing a CNN for Text Classification in TensorFlow

In this section we'll talk about the classification model.

The following chart shows roughly how the data flows in our final project that is implemented below.  
(by Kim, J. https://towardsdatascience.com/understanding-how-convolutional-neural-network-cnn-perform-text-classification-with-word-d2ee64b9dd0b)

![my_test_image](Picture/p2.jpeg)

## Extract Data From Facebook

##### import requests
import time
import json
import random
import csv
import re
import string
import time
import os
import pandas as pd


Create data by using facebook graph API
https://developers.facebook.com/docs/graph-api

You will have to use a API token to access to facebook graph API. Read this for how to get token:  
https://towardsdatascience.com/how-to-use-facebook-graph-api-and-extract-data-using-python-1839e19d6999

In [2]:
token = 'Your token here'

Function to extract data from facebook api

In [3]:
def reqest_facebook(req):
    url = 'https://graph.facebook.com/v2.12/' + req + '?fields=posts.limit(1){picture,message,created_time}&access_token=' + token
    r = requests.get(url)
    return r

This following code will go through all posts of an account and return the text with a lebel.

In [4]:
def shot_target(target):
    result = reqest_facebook(target).json()

    translation = str.maketrans("","", string.punctuation)
    try:
        #first post
        message = result['posts']['data'][0]['message']
        message = clean_str(message)
        message = message.translate(translation)
        time = result['posts']['data'][0]['created_time']
        pictureLink = result['posts']['data'][0]['picture']

#        print("Downloading image"+str(index))
#        pd.download_picture(pictureLink,pic_path)
#        print("Analysing image"+str(index))
#        picInfo=pp.detect_labels(pic_path)
#        print("Deleting image"+str(index))
#        os.remove(pic_path)
#         print(len(data))
        data.append([message,target])
    except KeyError as e:
        if e.args[0] == "next":
            print("reach the end")
        else:
            print("connect error, skip this post")
        
    #second post
    try:
        r=requests.get(result['posts']['paging']['next'])
        result = r.json()
        new_messg = result['data'][0]['message']
        new_messg = clean_str(new_messg)
        new_messg = new_messg.translate(translation)
        new_time = result['data'][0]['created_time']
        new_picture = result['data'][0]['picture']

#        print("Downloading image"+str(index))
#        pd.download_picture(new_picture,pic_path)
#        print("Analysing image"+str(index))
#        picInfo=pp.detect_labels(pic_path)
#        print("Deleting image"+str(index))
#        os.remove(pic_path)
#         print(len(data))   
        data.append([new_messg,target])
    except KeyError as e:
        if e.args[0] == "next":
            print("reach the end")
        else:
            print("connect error, skip this post")
            
        
    while True:
        try:
            r = requests.get(result['paging']['next'])
            result = r.json()
            new_messg = result['data'][0]['message']
            new_messg = clean_str(new_messg)
            new_messg = new_messg.translate(translation)
            new_time = result['data'][0]['created_time']
            new_picture = result['data'][0]['picture']

#            print("Downloading image"+str(index))
#            pd.download_picture(new_picture,pic_path)
#            print("Analysing image"+str(index))
#            picInfo=pp.detect_labels(pic_path)
#            print("Deleting image"+str(index))
#            os.remove(pic_path)
#             print(len(data))
            data.append([new_messg,target])
            if len(data) > datasize:
                print("reach the limit")
                print(target + " load data end!")
                break
        except KeyError as e:
            if e.args[0] == "next":
                print("reach the end")
                break
            else:
                print("connect error, skip this post")
            continue

We also can read the picture from the post and transform the picture to keywords bu using Google vision API:
https://cloud.google.com/vision/

Google vision API will process a picture and return a list of keywords

In [5]:
# import io
# import os
# import csv

# from google.cloud import vision
# from google.cloud.vision import types

    
# def detect_labels(pic_path):
#     client = vision.ImageAnnotatorClient()

#     with io.open(pic_path, 'rb') as image_file:
#         content = image_file.read()
#     image = types.Image(content=content)

#     response = client.label_detection(image=image)
#     labels = response.label_annotations
    
#     result=[]
#     for label in labels:
#         result.append(label.description)
        
#     return result

# def csv_writer(data, path):
#     with open(path, "w", newline='') as csv_file:
#         writer = csv.writer(csv_file, delimiter=',')
#         writer.writerow(data)

Because the facebook picture url is in a special format, Google API can not use it directly.
So, we will download the picture and prcess it then delete it.

In [6]:
# import requests
# def download_picture(uri,path):
#     with open(path, 'wb') as handle:
#             response = requests.get(uri, stream=True)

#             if not response.ok:
#                 print (response)

#             for block in response.iter_content(1024):
#                 if not block:
#                     break

#                 handle.write(block)

However, the model wo build on CNN does not need the keyword of picture for now. Besides, processing the picture take a very long time.
So, we will keep the picture processing code in our project, but we are not going to use it at this point.       

Code for pre-process text:
Clean the post text

In [7]:
def clean_str(string):
    """
    from: https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()


Function to extract the data and save it to a csv file

In [8]:
def run_facebookdata(target, result_path):
    print(target)
    shot_target(target)
            
    myFile = open(result_path, 'w', newline='', encoding='utf-8')
    with myFile:
        writer = csv.writer(myFile)
        writer.writerow(['Text','label'])
        for row in data:
            writer.writerow(row)
    print("Writing complete")

Begin to run.

@datasize: the size of post of every official account

In [9]:
datasize = 10
# pic_path = 'temp/tempPic.jpg'
data = []
target__list= ['steam', 'nytimes', 'NASA', 'linkedin', 'bbcnews']

for accout in target__list:
    result_path = 'Data/' + accout + '.csv'
    run_facebookdata(accout, result_path)
    data.clear()
    

steam
reach the limit
steam load data end!
Writing complete
nytimes
reach the limit
nytimes load data end!
Writing complete
NASA
connect error, skip this post
reach the limit
NASA load data end!
Writing complete
linkedin
reach the limit
linkedin load data end!
Writing complete
bbcnews
reach the limit
bbcnews load data end!
Writing complete


Function to run test dataset

In [11]:
datasize = 10
# pic_path = 'temp/tempPic.jpg'
data = []
target__list= ['steam', 'nytimes', 'NASA', 'linkedin', 'bbcnews']

for accout in target__list:
    result_path = 'Predict/' + accout + '.csv'
    run_facebookdata(accout, result_path)
    data.clear()

steam
reach the limit
steam load data end!
Writing complete
nytimes
reach the limit
nytimes load data end!
Writing complete
NASA
connect error, skip this post
reach the limit
NASA load data end!
Writing complete
linkedin
reach the limit
linkedin load data end!
Writing complete
bbcnews
reach the limit
bbcnews load data end!
Writing complete


let's look at our dataset

In [1]:
!ls Data

NASA.csv     bbcnews.csv  linkedin.csv nytimes.csv  steam.csv


In [3]:
df = pd.read_csv('Data/NASA.csv')
df.describe()

,Text,label
count,3559,3559
unique,3516,1
top,get the hottest ticket of the summer send your...,NASA
freq,5,3559


In [4]:
df.head()

,Text,label
0,launching monday nasa s transiting exoplanet ...,NASA
1,the stars above are rich with planetary compan...,NASA
2,once launched our transiting exoplanet survey...,NASA
3,on monday nasa s transiting exoplanet survey ...,NASA
4,we re live from nasa s kennedy space center on...,NASA


## Training model

If you running on Spark, You will need to add tensorflow library to your cluster:   
1.click databricks on the top right.    
2.click Libary under New.   
3.select Upload Python Egg or PyPI    
4.fill "tensorflow" in Pypi Name    
5.Inatall Libary

If you running on local, you can check https://www.tensorflow.org/install/ to install Tensorflow




Now we have our data. The next step is building a CNN model on tensorflow.

In [2]:
import numpy as np
import re
import itertools
import csv
from numpy import array
from collections import Counter
import pandas as pd
from tensorflow.contrib import learn
import tensorflow as tf
import time
import datetime
import os
import sys

Then we need to load our data and labels  
We have five classes in this demo, so the label will be a 5 digits vector like [0,0,0,0,1] and [0,0,0,1,0].
A CNN model with five classes required a long time to train. If you want a quick demo, you can use the load_data_and_labels3

In [3]:
#load 5 classes of data
def load_data_and_labels5(data_path):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    x=[]
    y=[]
    # Load data from files
    for subdir, dirs, files in os.walk(data_path):
        for file in files:
            file_name = data_path + file
            ## check if file is hidden file
            if not file.startswith('.'):
                with open(file_name) as csvfile:
                    readCSV = csv.reader(csvfile, delimiter=',')
                    next(readCSV)
                    for row in readCSV:
                        x.append(row[0])
                        if row[1] == 'bbcnews':
                            y.append([0,0,0,0,1])
                        elif row[1] == 'linkedin':
                            y.append([0,0,0,1,0])
                        elif row[1] == 'NASA':
                            y.append([0,0,1,0,0])
                        elif row[1] == 'nytimes':
                            y.append([0,1,0,0,0])
                        elif row[1] == 'steam':
                            y.append([1,0,0,0,0])					
    y = array (y)
    print('5 Category, Data loaded!')
    return [x, y]

In [4]:
#load 3 classes of data	
def load_data_and_labels3(data_path):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    x=[]
    y=[]
    # Load data from files
    for subdir, dirs, files in os.walk(data_path):
        for file in files:
            file_name = data_path+file
            print(file_name)
            if not file.startswith('.'):
                with open(file_name) as csvfile:
                    readCSV = csv.reader(csvfile, delimiter=',')
                    next(readCSV)
                    for row in readCSV:
                        if row[1] =='bbcnews' or row[1] =='linkedin' or row[1] =='NASA' :
                            x.append(row[0])
                            if row[1] == 'bbcnews':
                                y.append([0,0,1])
                            elif row[1] == 'linkedin':
                                y.append([0,1,0])
                            elif row[1] == 'NASA':
                                y.append([1,0,0])				
    y = array (y)
    print('3 Category, Data loaded!')
    return [x, y]

Config parameter：      
act_function: 0 for relu, 1 for elu      
cost_function: 0 for softmax_cross_entropy, 1 for sigmoid_cross_entropy_with_logits     
gradient: 0 for Adam, 1 for Adagrad   
epoches: number of epochs    
architecture: embedding dimensions for word2vertor and number of filters for CNN    
filter_sizes: size for each filters in CNN   
init: initialization config, True for uniform, False for random

In [5]:
#change config
act_function = 0
cost_function = 0
epochs=1
gradient = 0
architecture = [128,128]
filter_sizes = "3,4,5"
init = False

other config

In [6]:
# Data loading params
dev_sample_percentage = .1
data_path = "Data/"

# Model Hyperparameters
embedding_dim = architecture[0]
filter_sizes = filter_sizes
num_filters = architecture[1]
dropout_keep_prob =  0.5
l2_reg_lambda = 0.0

# Training parameters
batch_size =  64
num_epochs = epochs
evaluate_every = 100
checkpoint_every = 1
num_checkpoints = 5
# Misc Parameters
allow_soft_placement = True
log_device_placement = False

Loading data and show the data size

In [17]:
x_text,y=load_data_and_labels5(data_path)

5 Category, Data loaded!


Building vocabulary for word2vector

In [18]:
# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


Shuffle

In [19]:
# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

90% data for traning, 10% for evaluation

In [20]:
dev_sample_index = -1 * int(dev_sample_percentage * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]


In [21]:
del x, y, x_shuffled, y_shuffled

print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

Vocabulary Size: 771
Train/Dev split: 50/5


The CNN model we are using form http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/      
This model by DENNY BRITZ is licensed under the Apache License Version 2.0 https://www.apache.org/licenses/LICENSE-2.0

In [26]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      #Change statement: we add few parameters to change the model config
      embedding_size, filter_sizes, num_filters, act_function, cost_function, init, l2_reg_lambda):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

    # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")

                # Apply nonlinearity
                #Change statement: we add act_function to switch between relu and elu
                if act_function==0:
                    h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                elif act_function==1:
                    h = tf.nn.elu(tf.nn.bias_add(conv, b), name="elu")

                # relu: make all negtive number in feature map to 0, leave all postive number

                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features

        #flatting
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                #Change statement: we add uniform to switch between random and uniform
                initializer=tf.contrib.layers.xavier_initializer(uniform=init))
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")
  # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            #Change statement: we add cost_function to switch between softmax_cross_entropy_with_logits and sigmoid_cross_entropy_with_logits
            if cost_function == 0:
                losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            elif cost_function == 1:
                losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")


method to separate for each batch

In [7]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

The training progress

In [28]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=allow_soft_placement,
      log_device_placement=log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        #create cnn using the config 
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=embedding_dim,
            filter_sizes=list(map(int, filter_sizes.split(","))),
            num_filters=num_filters,
            act_function=act_function,
            cost_function=cost_function,
            init=init,
            l2_reg_lambda=l2_reg_lambda)
          
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        if gradient == 0:
            optimizer = tf.train.AdamOptimizer(1e-3)
        elif gradient ==1:
            optimizer = tf.train.AdagradOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        
         # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)
        
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.join(os.path.curdir, "Runs", timestamp)
        print("Writing to {}\n".format(out_dir))
        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)
        
        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)
        print("Train Summaries {}\n".format(train_summary_dir))
#         dbutils.fs.put(train_summary_dir, tf.summary)
#         dbutils.fs.put(train_summary_dir, sess.graph)
        
        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)
        print("Dev summaries {}\n".format(dev_summary_dir))
        
        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.join(out_dir, "checkpoints")
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        print("checkpoint_dir {}\n".format(checkpoint_dir))
        print("checkpoint_prefix {}\n".format(checkpoint_prefix))
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)
    # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))
        
        # Initialize all variables
        sess.run(tf.global_variables_initializer())
        
        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)
            
        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
                
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
                
 # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), batch_size, num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))


INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

## Evaluate Model

In [8]:
# find the lastest folder for model

folderList = []


for dirs in os.listdir('Runs/'):
    dirstring = 'Runs/' + dirs
    if os.path.isdir(dirstring):
        folderList.append(dirstring)

folderString = folderList[len(folderList) - 1]
folderString = folderString
checkpointString = folderString + '/checkpoints' 
print('folder location: ' + folderString)
print('model location: ' + checkpointString)

folder location: Runs/1524191220
model location: Runs/1524191220/checkpoints


In [9]:
# Data Parameters
pre_datapath = './Predict/'

# Eval Parameters
batch_size = 64
checkpoint_dir = folderString
eval_train = False
checkpoint_path = './' + checkpointString


# Misc Parameters
allow_soft_placement = True
log_device_placement = False


In [10]:
# load evaluation data

In [11]:
if eval_train:
    x_raw = ["a masterpiece four years in the making", "everything is off."]
    y_test = [1, 0]
else:
    #x_raw, y_test = cnn_model.load_data_and_labels5(FLAGS.path)
	#if you are using load_data_and_labels3 in train_model, use it here as well
    x_raw, y_test = load_data_and_labels5(pre_datapath)
    y_test = np.argmax(y_test, axis=1)
    print('Predict data loaded!')

5 Category, Data loaded!
Predict data loaded!


Map data into vocabulary

In [12]:
vocab_path = os.path.join(checkpoint_dir, "vocab")
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)
x_test = np.array(list(vocab_processor.transform(x_raw)))

Instructions for updating:
Please use tensorflow/transform or tf.data.


Read model parameters

In [13]:
checkpoint_file = tf.train.latest_checkpoint(checkpoint_path)
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=allow_soft_placement,
      log_device_placement=log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        # Generate batches for one epoch
        batches = batch_iter(list(x_test), batch_size, 1, shuffle=False)

        # Collect the predictions here
        all_predictions = []

        for x_test_batch in batches:
            batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
            all_predictions = np.concatenate([all_predictions, batch_predictions])

INFO:tensorflow:Restoring parameters from ./Runs/1524191220/checkpoints/model-1200


Print accuracy

In [14]:
if y_test is not None:
    correct_predictions = float(sum(all_predictions == y_test))
    print("Total number of test examples: {}".format(len(y_test)))
    print("Accuracy: {:g}".format(correct_predictions/float(len(y_test))))

Total number of test examples: 505
Accuracy: 0.891089


Save the evaluation result to a csv

In [15]:
# decode y_test and all_predictions
text_label = []
for i in range(len(y_test)):
    if y_test[i] == 0:
        text_label.append('steam')
    if y_test[i] == 1:
        text_label.append('nytimes')
    if y_test[i] == 2:
        text_label.append('NASA')
    if y_test[i] == 3:
        text_label.append('linkedin')
    if y_test[i] == 4:
        text_label.append('bbcnews')

prediction_label = []
for i in range(len(all_predictions)):
    if all_predictions[i] == 0:
        prediction_label.append('steam')
    if all_predictions[i] == 1:
        prediction_label.append('nytimes')
    if all_predictions[i] == 2:
        prediction_label.append('NASA')
    if all_predictions[i] == 3:
        prediction_label.append('linkedin')
    if all_predictions[i] == 4:
        prediction_label.append('bbcnews')

predictions_human_readable = np.column_stack((np.array(x_raw), prediction_label, text_label))
out_path = os.path.join(checkpoint_dir, "..", "prediction.csv")
print("Saving evaluation to {0}".format(out_path))

summary_path = os.path.join(checkpoint_dir, 'summaries')
with open(out_path, 'w') as f:
    csv.writer(f).writerows(predictions_human_readable)
    
print('Summary path is: ' + summary_path)

Saving evaluation to Runs/1524191220/../prediction.csv
Summary path is: Runs/1524191220/summaries


use below command to see visual summary:

!tensorboard --logdir summary_path

i.e: !tensorboard --logdir Runs/1524166522/summaries

# Result

In [37]:
from IPython.display import HTML, display

For dataset, we have 5 official account, total post number is 18293

After 5 epchos, 1290 steps, the best accuracy in validation setis 89.10%

<table><tr><td><img src='Picture/r1.jpeg'></td><td><img src='Picture/r2.jpeg'></td></tr></table>

And for test set, the accuracy is: 89.10%

## Reference

1.Britz, D. (2015, December 11). Implementing a CNN for Text Classification in TensorFlow. Retrieved from http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/  
2.Britz, D. (2015, November 7). Understanding Convolutional Neural Networks for NLP. Retrieved from http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/  
3.Kim, J. (2017, December 2). Understanding how Convolutional Neural Network (CNN) perform text classification with word embeddings. Retrieved from https://towardsdatascience.com/understanding-how-convolutional-neural-network-cnn-perform-text-classification-with-word-d2ee64b9dd0b  
4.A Beginner's Guide to Deep Convolutional Neural Networks (CNNs). (n.d.). Retrieved from https://deeplearning4j.org/convolutionalnetwork.html  
5.Word2Vec word embedding tutorial in Python and TensorFlow. (2017, July 21). Retrieved from http://adventuresinmachinelearning.com/word2vec-tutorial-tensorflow/  
6.Python TensorFlow Tutorial – Build a Neural Network. (2017, April 8). Retrieved from http://adventuresinmachinelearning.com/python-tensorflow-tutorial/    
7.Gaussic. (2018, February 19). CNN-RNN中文文本分类，基于tensorflow. Retrieved from https://github.com/gaussic/text-classification-cnn-rnn

## License

This notebook and all related works are under MIT License: https://opensource.org/licenses/MIT